# Feature Selection

## Why feature selection?

Feature selection, or *variable selection*, is an often used technique in machine learning. It is a process of selecting subset of highly relevant data to benefit modelling in many ways. Such as,

- Reduce the complexity, that is, improve the efficiency of training [1].

- Improve the prediction accuracy [1,2].




## Find the top *k* features most correlated to Saleprice (Naïve method with Parallel)


### Data Description

(House price data train.csv from https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)

The data have 1460 rows and 81 columns. The following code will print out the data description in detail.

### Load required libraries

In [2]:
import pandas as pd
import numpy as np

import time
import psutil
import os,gc
import scipy.stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed


### Define functions for data preprocessing

In [8]:

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Memory in MB

def compute_correlation(i, X_numeric, y):
    """Helper function to compute Pearson correlation for a single feature."""
    valid = (~np.isnan(X_numeric[:, i]) & ~np.isnan(y))
    if valid.sum() > 1:
        return np.abs(scipy.stats.pearsonr(X_numeric[:, i][valid], y[valid]).statistic)
    return 0

def parallel_feature_selection(X, y, k=20, n_jobs=2):
    """
    Parallelized feature selection using Pearson correlation.
    
    Parameters:
    - X: DataFrame with features
    - y: Target variable (Series or array)
    - k: Number of top features to select
    - n_jobs: Number of parallel jobs (-1 uses all available cores)
    
    Returns:
    - List of selected feature names
    """

    # Select only numerical columns and fill missing values with median values of the columns
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    # Convert to numpy array for faster processing
    X_numeric_array = X_numeric.to_numpy()
    y_array = y.to_numpy() if isinstance(y, pd.Series) else y

    # Compute correlations for numerical features only
    n_features = X_numeric.shape[1]
    correlations = Parallel(n_jobs=n_jobs, backend='threading')(
        delayed(compute_correlation)(i, X_numeric_array, y_array) for i in range(n_features)
    )

    # Convert correlations to numpy array and get top k indices
    correlations = np.array(correlations)

    top_k_indices = np.argsort(correlations)[-k:]
    selected_features = X_numeric.columns[top_k_indices].tolist()
    #mem_use_after = get_memory_usage()
    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")
    #print(f"memory use increased {(mem_use_after - mem_use_before):.2f} MB" )
    return selected_features

def load_and_evaluate(file_path, k=20, n_runs = 100, n_jobs=2):
    # Initialize lists to store metrics
    memory_before_list = []
    memory_after_list = []
    memory_increase_list = []
    runtime_list = []
    mse_list = []

    #load data
    data = pd.read_csv(file_path)
    # split Sale price and other columns
    X = data.drop(columns=['SalePrice', 'Id'])
    y = data['SalePrice']
    # Handle missing values for numerical columns 
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    X_categorical = X.select_dtypes(exclude=[np.number])
    # Combine numerical and categorical columns
    X = pd.concat([X_numeric, X_categorical], axis=1)
    
    # Verify no NaNs remain 
    if X.isna().any().any():
        X = X.fillna(0)  # Fill any remaining NaNs with 0 (e.g., for edge cases)
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Monte Carlo method for testing mem use and runtime
    print(f"Running naive method {n_runs} times...")
    for run in range(n_runs):
        # clear memory before each run
        gc.collect() 
        mem_use_before = get_memory_usage()
        #print(f"Memory before naive selection: {mem_use_before:.2f} MB")

        start_time = time.time()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+run)
        selected_features = parallel_feature_selection(X_train, y_train, k, n_jobs)
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)

        mem_use_after = get_memory_usage()
        end_time = time.time()
        # Store metrics
        memory_before_list.append(mem_use_before)
        memory_after_list.append(mem_use_after)
        memory_increase_list.append(mem_use_after - mem_use_before)
        runtime_list.append(end_time - start_time)
        mse_list.append(mse)

    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")

    #print(f"Naive feature selection took {time.time() - start_time} seconds")
    
    # Ensure selected features are NaN-free
    #X_train_selected = X_train[selected_features]
    #X_test_selected = X_test[selected_features]
    
    #model = LinearRegression()
    #model.fit(X_train_selected, y_train)
    #y_pred = model.predict(X_test_selected)
    #mse = mean_squared_error(y_test, y_pred)
    #print(f"Mean Squared Error with {k} features: {mse}")

    # Compute statistics
    stats = {
        'memory_before_mean (MB)': np.mean(memory_before_list),
        'memory_before_std': np.std(memory_before_list),
        'memory_after_mean (MB)': np.mean(memory_after_list),
        'memory_after_std': np.std(memory_after_list),
        'memory_increase_mean (MB)': np.mean(memory_increase_list),
        'memory_increase_std': np.std(memory_increase_list),
        'runtime_mean (Seconds)': np.mean(runtime_list),
        'runtime_std': np.std(runtime_list),
        'MSE_mean': np.mean(mse_list),
        'MSE_std': np.std(mse_list)

    }
    
    return selected_features, mse, stats

def generate_report(stats, method="Naive"):
    report = f"# {method} Method Statistical Report\n\n"
    report += "## Summary Statistics\n\n"
    report += "| Metric | Mean | Standard Deviation |\n"
    report += "|--------|------|--------------------|\n"
    report += f"| Memory Before (MB) | {stats['memory_before_mean (MB)']:.2f} | {stats['memory_before_std']:.2f} |\n"
    report += f"| Memory After (MB) | {stats['memory_after_mean (MB)']:.2f} | {stats['memory_after_std']:.2f} |\n"
    report += f"| Memory Increase (MB) | {stats['memory_increase_mean (MB)']:.2f} | {stats['memory_increase_std']:.2f} |\n"
    report += f"| Runtime (seconds) | {stats['runtime_mean (Seconds)']:.6f} | {stats['runtime_std']:.6f} |\n"
    report += f"| MSE | {stats['MSE_mean']:.2f} | {stats['MSE_std']:.2f} |\n"

    return report

### Main process of evaluation with Monte Carlo

In [10]:

if __name__ == "__main__":
    gc.collect()
    file_path = "data/train.csv"
    n_runs = 500
    n_jobs=2
    selected_features, mse, stats = load_and_evaluate(file_path,  k=20, n_runs=n_runs, n_jobs=n_jobs)
    #print(f"Selected features (last run): {selected_features}")
    report = generate_report(stats, method="Naive")
    with open("naive_stats_report.md", "w") as f:
        f.write(report)
    print("Naive method report generated: naive_stats_report.md")
    print("\nSummary Statistics:")
    for key, value in stats.items():
        print(f"{key}: {value:.6f}")
    #mem_use_after = get_memory_usage()
    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")
    #print(f"memory use increased {(mem_use_after - mem_use_before):.2f} MB" )
    print(f"Selected features: {selected_features}")

Running naive method 500 times...
Naive method report generated: naive_stats_report.md

Summary Statistics:
memory_before_mean (MB): 188.491328
memory_before_std: 6.470636
memory_after_mean (MB): 188.565937
memory_after_std: 6.373005
memory_increase_mean (MB): 0.074609
memory_increase_std: 0.242563
runtime_mean (Seconds): 0.062643
runtime_std: 0.005848
MSE_mean: 1495907264.707669
MSE_std: 668916768.381671
Selected features: ['LotArea', 'HalfBath', '2ndFlrSF', 'WoodDeckSF', 'LotFrontage', 'OpenPorchSF', 'BsmtFinSF1', 'Fireplaces', 'MasVnrArea', 'GarageYrBlt', 'YearRemodAdd', 'YearBuilt', 'TotRmsAbvGrd', 'FullBath', '1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']


### Discussion

Parallel method with Monte Carlo introduces overhead for 

- parallelization: process creation and communication

- Monte Carlo method introduces cumulative parallelization overhead

Therefore, for small tasks like this, parallel method does not necessarily faster than the loop-based single thread job.

### Try with a larger dataset

# run this only once
def generate_synthetic_dataset(n_rows=100000, n_features=500, file_path="synthetic_data.csv"):
    print(f"Generating synthetic dataset with {n_rows} rows and {n_features} features...")
    np.random.seed(42)
    
    # Generate numerical features
    X = np.random.randn(n_rows, n_features)
    print(f"X shape: {X.shape}")  # Debug: Confirm 2D shape (n_rows, n_features)
    
    # Generate target variable with some correlated features
    weights = np.random.uniform(0.1, 1.0, 20)  # 20 features with significant correlation
    correlated_indices = np.random.choice(n_features, 20, replace=False)
    print(f"Correlated indices: {correlated_indices[:5]}...")  # Debug: Show first few indices
    y = np.zeros(n_rows)
    for idx, weight in zip(correlated_indices, weights):
        column = X[:, idx]  # Access column
        print(f"Column {idx} shape: {column.shape}")  # Debug: Confirm 1D shape (n_rows,)
        y += weight * column
    y += np.random.randn(n_rows) * 0.1  # Add noise
    print(f"y shape: {y.shape}")  # Debug: Confirm 1D shape (n_rows,)
    
    # Create DataFrame
    columns = [f"Feature_{i}" for i in range(n_features)]
    data = pd.DataFrame(X, columns=columns)
    data["SalePrice"] = y
    data["Id"] = range(1, n_rows + 1)
    
    # Introduce some missing values (5% of numerical data)
    mask = np.random.random((n_rows, n_features)) < 0.05
    print(f"Mask shape: {mask.shape}")  # Debug: Confirm 2D shape (n_rows, n_features)
    data.iloc[:, :n_features][mask] = np.nan  # Apply mask to numerical features only
    
    # Save to CSV
    data.to_csv(file_path, index=False)
    print(f"Dataset saved to {file_path}")
    return file_path
file_path = generate_synthetic_dataset(n_rows=100000, n_features=500)

In [6]:
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Memory in MB

def naive_feature_selection(X, y, k=20):
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    n_features = X_numeric.shape[1]
    correlations = np.zeros(n_features)
    for i in range(n_features):
        valid = (~np.isnan(X_numeric.iloc[:, i]) & ~np.isnan(y))
        if valid.sum() > 1:
            correlations[i] = np.abs(scipy.stats.pearsonr(X_numeric.iloc[:, i][valid], y[valid]).statistic)
        else:
            correlations[i] = 0
    top_k_indices = np.argsort(correlations)[-k:]
    return X_numeric.columns[top_k_indices].tolist()

def compute_correlation(i, X_numeric, y):
    valid = (~np.isnan(X_numeric[:, i]) & ~np.isnan(y))
    if valid.sum() > 1:
        return np.abs(scipy.stats.pearsonr(X_numeric[:, i][valid], y[valid]).statistic)
    return 0

def parallel_feature_selection(X, y, k=20, n_jobs=-1):
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    X_numeric_array = X_numeric.to_numpy()
    y_array = y.to_numpy() if isinstance(y, pd.Series) else y
    n_features = X_numeric_array.shape[1]
    correlations = Parallel(n_jobs=n_jobs)(
        delayed(compute_correlation)(i, X_numeric_array, y_array) for i in range(n_features)
    )
    correlations = np.array(correlations)
    top_k_indices = np.argsort(correlations)[-k:]
    return X_numeric.columns[top_k_indices].tolist()


def load_and_evaluate(file_path, k=20, n_runs=10, n_jobs=-1):
    metrics = {
        "naive": {"memory_before": [], "memory_after": [], "memory_increase": [], "runtime": [], "mse": []},
        "parallel": {"memory_before": [], "memory_after": [], "memory_increase": [], "runtime": [], "mse": []}
    }
    
    data = pd.read_csv(file_path)
    X = data.drop(columns=["SalePrice", "Id"])
    y = data["SalePrice"]
    
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    X_categorical = X.select_dtypes(exclude=[np.number])
    X = pd.concat([X_numeric, X_categorical], axis=1)
    if X.isna().any().any():
        X = X.fillna(0)
    
    print(f"Running both methods {n_runs} times...")
    for run in range(n_runs):
        gc.collect()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+run)
        
        # Naive method
        mem_before = get_memory_usage()
        start_time = time.time()
        selected_features_naive = naive_feature_selection(X_train, y_train, k)
        X_train_selected = X_train[selected_features_naive]
        X_test_selected = X_test[selected_features_naive]
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mem_after = get_memory_usage()
        runtime = time.time() - start_time
        metrics["naive"]["memory_before"].append(mem_before)
        metrics["naive"]["memory_after"].append(mem_after)
        metrics["naive"]["memory_increase"].append(mem_after - mem_before)
        metrics["naive"]["runtime"].append(runtime)
        metrics["naive"]["mse"].append(mse)
        
        # Parallel method
        gc.collect()
        mem_before = get_memory_usage()
        start_time = time.time()
        selected_features_parallel = parallel_feature_selection(X_train, y_train, k, n_jobs)
        X_train_selected = X_train[selected_features_parallel]
        X_test_selected = X_test[selected_features_parallel]
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mem_after = get_memory_usage()
        runtime = time.time() - start_time
        metrics["parallel"]["memory_before"].append(mem_before)
        metrics["parallel"]["memory_after"].append(mem_after)
        metrics["parallel"]["memory_increase"].append(mem_after - mem_before)
        metrics["parallel"]["runtime"].append(runtime)
        metrics["parallel"]["mse"].append(mse)
    
    stats = {}
    for method in ["naive", "parallel"]:
        stats[method] = {
            "memory_before_mean (MB)": np.mean(metrics[method]["memory_before"]),
            "memory_before_std": np.std(metrics[method]["memory_before"]),
            "memory_after_mean (MB)": np.mean(metrics[method]["memory_after"]),
            "memory_after_std": np.std(metrics[method]["memory_after"]),
            "memory_increase_mean (MB)": np.mean(metrics[method]["memory_increase"]),
            "memory_increase_std": np.std(metrics[method]["memory_increase"]),
            "runtime_mean (Seconds)": np.mean(metrics[method]["runtime"]),
            "runtime_std": np.std(metrics[method]["runtime"]),
            "MSE_mean": np.mean(metrics[method]["mse"]),
            "MSE_std": np.std(metrics[method]["mse"])
        }
    
    return stats

def generate_comparison_report(stats):
    report = "# Feature Selection Method Comparison Report\n\n"
    report += "## Summary Statistics\n\n"
    for method in ["naive", "parallel"]:
        report += f"### {method.capitalize()} Method\n"
        report += "| Metric | Mean | Standard Deviation |\n"
        report += "|--------|------|--------------------|\n"
        report += f"| Memory Before (MB) | {stats[method]['memory_before_mean (MB)']:.2f} | {stats[method]['memory_before_std']:.2f} |\n"
        report += f"| Memory After (MB) | {stats[method]['memory_after_mean (MB)']:.2f} | {stats[method]['memory_after_std']:.2f} |\n"
        report += f"| Memory Increase (MB) | {stats[method]['memory_increase_mean (MB)']:.2f} | {stats[method]['memory_increase_std']:.2f} |\n"
        report += f"| Runtime (Seconds) | {stats[method]['runtime_mean (Seconds)']:.4f} | {stats[method]['runtime_std']:.4f} |\n"
        report += f"| MSE | {stats[method]['MSE_mean']:.2f} | {stats[method]['MSE_std']:.2f} |\n\n"
    
    runtime_diff = stats["naive"]["runtime_mean (Seconds)"] - stats["parallel"]["runtime_mean (Seconds)"]
    report += "## Key Comparison\n"
    report += f"- **Runtime Difference (Naive - Parallel)**: {runtime_diff:.4f} seconds\n"
    report += f"  - Positive means parallel is faster; negative means naive is faster.\n"
    report += f"- **MSE Difference (Naive - Parallel)**: {stats['naive']['MSE_mean'] - stats['parallel']['MSE_mean']:.2f}\n"
    report += f"  - Similar MSE values indicate both methods select comparable features.\n"
    
    return report

def main():
    try:
        file_path = "data/synthetic_data.csv"
        stats = load_and_evaluate(file_path, k=20, n_runs=10, n_jobs=-1)
        report = generate_comparison_report(stats)
        print(report)
        with open("comparison_report.md", "w") as f:
            f.write(report)
        print("Report saved to comparison_report.md")
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Running both methods 10 times...
# Feature Selection Method Comparison Report

## Summary Statistics

### Naive Method
| Metric | Mean | Standard Deviation |
|--------|------|--------------------|
| Memory Before (MB) | 1561.66 | 174.43 |
| Memory After (MB) | 1487.98 | 205.38 |
| Memory Increase (MB) | -73.68 | 273.52 |
| Runtime (Seconds) | 5.4579 | 0.1668 |
| MSE | 0.01 | 0.00 |

### Parallel Method
| Metric | Mean | Standard Deviation |
|--------|------|--------------------|
| Memory Before (MB) | 1488.53 | 204.84 |
| Memory After (MB) | 1336.93 | 296.75 |
| Memory Increase (MB) | -151.60 | 232.06 |
| Runtime (Seconds) | 2.7431 | 2.1951 |
| MSE | 0.01 | 0.00 |

## Key Comparison
- **Runtime Difference (Naive - Parallel)**: 2.7148 seconds
  - Positive means parallel is faster; negative means naive is faster.
- **MSE Difference (Naive - Parallel)**: 0.00
  - Similar MSE values indicate both methods select comparable features.

Report saved to comparison_report.md


## Reference

1. Gareth James; Daniela Witten; Trevor Hastie; Robert Tibshirani (2013). "An Introduction to Statistical Learning". *Springer*. p. 204.
2. Kratsios, Anastasis; Hyndman, Cody (2021). "NEU: A Meta-Algorithm for Universal UAP-Invariant Feature Representation". *Journal of Machine Learning Research*. 22 (92): 1–51